# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [50]:
# import libraries
import numpy as np
import pandas as pd
import sys
from sqlalchemy import create_engine
import nltk
import joblib

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import re

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as iPipeline

from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, recall_score, f1_score

from mismote import MLSMOTE

[nltk_data] Downloading package punkt to /home/salman1567/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/salman1567/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Part I: Loading the Dataset

In [51]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster.db', engine)
# X = df.iloc[:,1]
# Y = df.iloc[:,4:]

# We can drop child alone since there aren't any messages with that category.


#print(X.head())

### Method 1: Undersampling
In this method we will only take a subset of smaples from the entire dataset. Knowing this should be okay since most of the text data have similar text.


In [52]:


def underSampleDataSet(df, samples=200):
    df_resampled = []
    df.drop(['child_alone'], axis=1, inplace=True)
    for i in df.iloc[:,4:].columns:
        new_df = df[df[i]==1]
        print(new_df.shape)
        df_resampled.append(new_df.sample(200, replace=True))
    
    # Concating the 200 samples from each category into one and returning it.
    df = pd.concat(df_resampled)
    return df

df = underSampleDataSet(df)




(20042, 39)
(4513, 39)
(121, 39)
(10954, 39)
(2099, 39)
(1319, 39)
(725, 39)
(471, 39)
(863, 39)
(1684, 39)
(2956, 39)
(2342, 39)
(410, 39)
(606, 39)
(301, 39)
(880, 39)
(1204, 39)
(3464, 39)
(1709, 39)
(1212, 39)
(1345, 39)
(538, 39)
(159, 39)
(283, 39)
(120, 39)
(309, 39)
(1155, 39)
(7343, 39)
(2177, 39)
(2464, 39)
(282, 39)
(2461, 39)
(533, 39)
(1379, 39)
(5113, 39)


In [53]:
# check if they have been resampled

print(df.shape)

(7000, 39)


This dataset is imbalanced so we will try to attempt to merge some of the really small categories which have less than 200 samples. This does mean we lose some specificity with our model and this is an important consideration if this trade off is acceptable or not. It can be argued in some situations we don't need to know the exact specific disaster but rather the general category such as if it is infrastructural related we make sure we send out a team with tools and general capability to handle that kind of situation.

We will be merging buildings, electricity, tools, hospitals, shops, aid-centers, other_infrastructure all into `infrastructure_related`. The reason for doing this makes sense from a logical point of view since those are all infrastructure related and will typically mean we send out similar teams prepared for any kind of infrastructure related problems.

We will also be placing clothing, money, missing people, refugees and death category into `other_aid`



## Part II: Experimenting with Feature Engineering

Attempting to experiment with feature engineering to see if f1 score will improve by making the model more general and merging some of the indepdent features together.

In [54]:

def merge_features(df):
      # If any of these features are 1 than infrastructure_related should be 1 since we merging them in
      
      df.loc[(df['buildings']==1) |
            (df['electricity']==1) |
            (df['tools']==1) |
            (df['hospitals']==1) |
            (df['shops']==1) |
            (df['aid_centers']==1) |
            (df['other_infrastructure']==1), 'infrastructure_related'] = 1

      # Merging categories into other aid

      df.loc[(df['clothing']==1) |
            (df['money']==1) |
            (df['missing_people']==1) |
            (df['refugees']==1) |
            (df['death']==1), 'other_aid'] = 1

      # Merging floods, storm, cold and other_weather into weather_related

      df.loc[(df['floods']==1) |
            (df['storm']==1) |
            (df['cold']==1) |
            (df['other_weather']==1), 'weather_related'] = 1

      # for i in df.columns.unique()[7:]:
      #       print(df.columns.unique())
      #       if df[df[i]==1].shape[0] > 1000:
      #             pass
      #       else:
      #             df = df.drop(labels=[i])


      # Dropping the merged in features
      df = df.drop(labels=['buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
            'clothing', 'money', 'missing_people', 'refugees', 'death', 'floods', 'storm', 'cold', 'other_weather'], axis=1)

# merge_features(df)

#### Feature merging

Later when running the pipeline we see that this merging improves f1 score very slightly, lowering precision but improving recall thus overall with an improved f1 score. Since we are dealing with an imbalanced dataset we will prioritize recall and f1 score over precision.

In [55]:
# Making sure that the features successfuly merged into infrastructure_related
X = df.iloc[:,1]
Y = df.iloc[:,4:]
total_samples = Y.shape[0]
for i in Y.columns.unique():
    number_of_samples = Y[Y[i]==1].shape[0]
    print(i, number_of_samples, number_of_samples/total_samples,sep=" - ")

related - 7000 - 1.0
request - 2017 - 0.28814285714285715
offer - 252 - 0.036
aid_related - 6023 - 0.8604285714285714
medical_help - 1483 - 0.21185714285714285
medical_products - 1195 - 0.1707142857142857
search_and_rescue - 817 - 0.11671428571428571
security - 589 - 0.08414285714285714
military - 595 - 0.085
water - 1103 - 0.15757142857142858
food - 1575 - 0.225
shelter - 1611 - 0.23014285714285715
clothing - 462 - 0.066
money - 517 - 0.07385714285714286
missing_people - 473 - 0.06757142857142857
refugees - 824 - 0.11771428571428572
death - 970 - 0.13857142857142857
other_aid - 1929 - 0.2755714285714286
infrastructure_related - 1998 - 0.2854285714285714
transport - 1161 - 0.16585714285714287
buildings - 1143 - 0.16328571428571428
electricity - 573 - 0.08185714285714285
tools - 343 - 0.049
hospitals - 489 - 0.06985714285714285
shops - 327 - 0.046714285714285715
aid_centers - 477 - 0.06814285714285714
other_infrastructure - 1088 - 0.15542857142857142
weather_related - 4209 - 0.601285714

In [56]:
# Do the same for other aid
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
776,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20763,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1251,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3969,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Part III: Tokenization of text data
### 2. Write a tokenization function to process your text data

In [57]:
def tokenize(text):
    
    clean_tokens = []
    
    # lowercasing and removing punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())
    tokens = word_tokenize(text)
    # Remove stop words + stem
    stemmer = PorterStemmer()
    
    
    stop_words = stopwords.words("english")
    clean_tokens = [ stemmer.stem(word) for word in tokens if word not in stop_words]

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [58]:
# build a transformer that scales
class OverSamplingTransformer(BaseEstimator, TransformerMixin):
    """A custom transformer to be used in the ML pipeline to handle the class imbalance that is present
    in this dataset.
    """
    
    def __init__(self, n_sample):
        self.n_sample = n_sample
        pass
    
    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        y = pd.DataFrame(y)
        return MLSMOTE(X, y, self.n_sample)
    
    def transform(self, X):
        pass

In [59]:
# Try other estimators to see if performance is better
# Implement oversampling using SVD of n components

# Number of featurs we want to reduce to so that we can then use this in oversampling transformer
def build_pipeline(n_features=33):
    pipeline = Pipeline([
    #('smt', SMOTE()),
    ('features', FeatureUnion([
        ('nltk_pipeline', Pipeline([
                ('count-vector', CountVectorizer(tokenizer=tokenize)),
                ('tf-idf', TfidfTransformer())   
        ])),
        
        
        
    ])),
     #('tf-idf', TfidfVectorizer()),
    #('SVD', TruncatedSVD(n_components=n_features)),
    #('smote', OverSamplingTransformer(n_sample=1000)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
    ])
    
    return pipeline

pipeline = build_pipeline()




### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
pipeline.fit(X_train, y_train)


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('nltk_pipeline',
                                                 Pipeline(steps=[('count-vector',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7f403c40b7f0>)),
                                                                 ('tf-idf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=-1))])

In [61]:
print(X_train)

11074    There 's a storm on the ocean and it 's moving...
16803    [That] is why we make the distinction between ...
2710     2010 now I am sleeping on the streets. Please ...
2167     our neighborhood, the population is in great n...
1282     We are the people of Gran gwav, we are asking ...
                               ...                        
18786    Three cases of suspected diphtheria (in Gahri ...
15039    Ideally under normal conditions teams would be...
11213    @laurenkoslow I 'm going trough withdrawals .....
7616     We need water in 57 to serve us thereas 100 fa...
13631    Army troops with their high platform vehicles,...
Name: message, Length: 4690, dtype: object


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [62]:
col_names = Y.columns.unique()
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=col_names))
    



                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      2310
               request       0.91      0.80      0.85       675
                 offer       0.97      0.85      0.90        72
           aid_related       0.90      0.99      0.95      2008
          medical_help       0.92      0.63      0.75       515
      medical_products       0.98      0.67      0.80       408
     search_and_rescue       0.97      0.66      0.79       266
              security       0.97      0.53      0.69       214
              military       0.91      0.59      0.71       211
                 water       0.93      0.84      0.88       379
                  food       0.95      0.89      0.92       519
               shelter       0.93      0.77      0.84       512
              clothing       0.97      0.77      0.86       151
                 money       0.97      0.59      0.73       174
        missing_people       0.98      

In [63]:
# Checking what hyper parameters can be tuned
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__nltk_pipeline', 'features__nltk_pipeline__memory', 'features__nltk_pipeline__steps', 'features__nltk_pipeline__verbose', 'features__nltk_pipeline__count-vector', 'features__nltk_pipeline__tf-idf', 'features__nltk_pipeline__count-vector__analyzer', 'features__nltk_pipeline__count-vector__binary', 'features__nltk_pipeline__count-vector__decode_error', 'features__nltk_pipeline__count-vector__dtype', 'features__nltk_pipeline__count-vector__encoding', 'features__nltk_pipeline__count-vector__input', 'features__nltk_pipeline__count-vector__lowercase', 'features__nltk_pipeline__count-vector__max_df', 'features__nltk_pipeline__count-vector__max_features', 'features__nltk_pipeline__count-vector__min_df', 'features__nltk_pipeline__count-vector__ngram_range', 'features__nltk_pipeline__count-vector__preprocessor', 'features__n

`TODO`: Consider to remove child_alone since there are none of those and also consider oversampling since some of the disaster categories have a low amount of samples.

In [64]:
# we can consider dropping any columns that have zero rows associated
def drop_columns_with_low_samples():
    for i in df.columns.unique():
        if sum(df[i]) < 200:
            df = df.drop(columns=[i])
        


### 6. Improve your model
Use grid search to find better parameters. 

In [65]:
parameters = {'clf__estimator__n_estimators': [10,50,100,150],
              'clf__n_jobs': [-1], 
              'clf__estimator__n_jobs': [-1]}

cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=col_names))

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      2310
               request       0.93      0.85      0.89       677
                 offer       1.00      0.72      0.84        68
           aid_related       0.90      0.99      0.94      1974
          medical_help       0.92      0.68      0.78       487
      medical_products       0.98      0.70      0.82       410
     search_and_rescue       0.99      0.68      0.81       305
              security       0.99      0.67      0.80       200
              military       0.98      0.65      0.78       202
                 water       0.98      0.83      0.90       354
                  food       0.96      0.86      0.91       503
               shelter       0.96      0.77      0.85       564
              clothing       0.99      0.75      0.85       166
                 money       0.98      0.66      0.79       190
        missing_people       0.99      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [66]:
# Check what params gridsearchcv has selected
#cv.get_params()
from sklearn.tree import DecisionTreeClassifier
# Trying a different model
def build_pipeline2(classifier):

    pipeline2 = Pipeline([
        #('smt', SMOTE()),
        ('count-vector', CountVectorizer(tokenizer=tokenize)),
        ('tf-idf', TfidfTransformer()),
        #('tf-idf', TfidfVectorizer()),
        ('clf', MultiOutputClassifier(classifier, n_jobs=-1))
    ])
    
    return pipeline2

def train_pipeline2(pipeline2, X, Y):
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.5, random_state=3)
    
    #for i in y_train2.columns:
    #    print(sum(y_train2[i]), sum(y_test2[i]))
    
    pipeline2.fit(X_train2, y_train2)
    col_names = Y.columns.unique()
    y_pred2 = pipeline2.predict(X_test2)
    print(classification_report(y_test2, y_pred2, target_names=col_names))
    
pipeline2 = build_pipeline2(DecisionTreeClassifier())
train_pipeline2(pipeline2, X, Y)

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      3500
               request       0.74      0.78      0.76       997
                 offer       0.58      0.76      0.66       122
           aid_related       0.92      0.92      0.92      3002
          medical_help       0.66      0.71      0.69       734
      medical_products       0.75      0.73      0.74       605
     search_and_rescue       0.60      0.64      0.62       403
              security       0.56      0.59      0.58       285
              military       0.72      0.75      0.73       308
                 water       0.83      0.84      0.83       536
                  food       0.90      0.86      0.88       781
               shelter       0.84      0.78      0.81       805
              clothing       0.84      0.81      0.83       243
                 money       0.66      0.75      0.71       256
        missing_people       0.60      

#### Evaluation of the DecisionTreeClassifier
Trying to DecisionTreeClassifier model it is close in terms of the randomforest model as expected since the random forest classifier is an ensemble method of DecisionTreeClassifier.

In [67]:
# Trying AdaBoostClassifier model
pipeline2 = build_pipeline2(AdaBoostClassifier())
train_pipeline2(pipeline2, X, Y)


                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      3500
               request       0.80      0.58      0.67       997
                 offer       0.47      0.29      0.36       122
           aid_related       0.89      0.95      0.92      3002
          medical_help       0.72      0.40      0.51       734
      medical_products       0.76      0.44      0.56       605
     search_and_rescue       0.54      0.23      0.33       403
              security       0.47      0.20      0.28       285
              military       0.71      0.42      0.52       308
                 water       0.81      0.80      0.80       536
                  food       0.88      0.78      0.83       781
               shelter       0.82      0.58      0.68       805
              clothing       0.85      0.63      0.72       243
                 money       0.72      0.52      0.60       256
        missing_people       0.66      

#### Evaluation of the AdaBoostClassifier
We can see from the classification report that all our evaluation metrics precision, recall and f1 score are worse with AdaBoostClassifier. Therefore we should go with using RandomForestClassifier.

### 9. Export your model as a pickle file

In [ ]:
# Export the model as a pickle file
joblib.dump(cv, 'classifier.pkl')

['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
model = joblib.load('classifier.pkl')

/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.war

In [ ]:
model.predict(["".join("we are more than 50 people sleeping on the street. Please hep us find tent, food.")])


array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [ ]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16452,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
13143,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25308,1,0,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6910,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21432,1,0,0,1,0,0,0,0,0,1,...,0,0,1,1,0,0,0,1,0,0
17497,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
22275,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test[0]

'Weather update - a cold front from Cuba that could pass over Haiti'